# Collecting data on decentralized exchanges using BitQuery GraphQL API

**First we define how to run a query using API key from BitQuery**

In [21]:
import requests
import pandas as pd
import yaml


global API_KEY
with open('api.yaml', 'r') as f:
    parsed_yaml = yaml.load(f, Loader=yaml.FullLoader)
    API_KEY = parsed_yaml['api']


def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': API_KEY}
    request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}.      {}'.format(request.status_code, query))


**The below query fetches the top decentralized exchanges based on their protocol and sorted on the number of trades**

In [22]:
query = """
{
  ethereum {
    dexTrades(options: {limit: 100, desc: "count"}) {
      count
      protocol
    }
  }
}
"""
result = run_query(query)  # Execute the query
print ('Result - {}'.format(result))

Result - {'data': {'ethereum': {'dexTrades': [{'count': 87670684, 'protocol': 'Uniswap v2'}, {'count': 14159336, 'protocol': 'Uniswap v3'}, {'count': 5911016, 'protocol': 'IDEX'}, {'count': 3736115, 'protocol': 'EtherDelta'}, {'count': 2603444, 'protocol': 'Balancer Pool Token'}, {'count': 2538183, 'protocol': 'Uniswap'}, {'count': 2020098, 'protocol': 'Zerox Exchange v2'}, {'count': 1046443, 'protocol': 'Bancor Network'}, {'count': 1023483, 'protocol': 'Kyber Network v3'}, {'count': 866731, 'protocol': 'Bancor Network v2'}, {'count': 768626, 'protocol': 'dYdX2'}, {'count': 743531, 'protocol': 'Zerox Exchange'}, {'count': 487296, 'protocol': 'Matching Market'}, {'count': 466484, 'protocol': 'Zerox Exchange v3'}, {'count': 328196, 'protocol': 'Curve'}, {'count': 328131, 'protocol': 'Kyber Network v4'}, {'count': 320733, 'protocol': 'Token.Store'}, {'count': 320505, 'protocol': 'Zerox Exchange v4'}, {'count': 279603, 'protocol': 'Mooniswap'}, {'count': 233873, 'protocol': 'One Inch Liqui

In [23]:
# Convert the result to a dataframe
df = pd.json_normalize(result['data']['ethereum']['dexTrades'])
print(df)

       count                 protocol
0   87670684               Uniswap v2
1   14159336               Uniswap v3
2    5911016                     IDEX
3    3736115               EtherDelta
4    2603444      Balancer Pool Token
5    2538183                  Uniswap
6    2020098        Zerox Exchange v2
7    1046443           Bancor Network
8    1023483         Kyber Network v3
9     866731        Bancor Network v2
10    768626                    dYdX2
11    743531           Zerox Exchange
12    487296          Matching Market
13    466484        Zerox Exchange v3
14    328196                    Curve
15    328131         Kyber Network v4
16    320733              Token.Store
17    320505        Zerox Exchange v4
18    279603                Mooniswap
19    233873  One Inch Liquidity Pool
20    181136                     Dodo
21     91447         Kyber Network v2
22     77242          DDEX Hydro v1.1
23     73696          DDEX Hydro v1.0
24     48304         AirSwap Exchange
25     31764

**This is a good place to start as it shows us the most popular decentralized exchanges we can now compare**

**Next lets find the top trades on decentralized exchanges in USD**

In [24]:
query = """
{
   ethereum {
    dexTrades(options: {limit: 10 desc: "tradeAmount"}
    date: {after: "2020-11-01"}) {
      transaction {
        hash
      }
      tradeIndex
      date{date}

      buyAmount
      buyAmountInUsd:buyAmount(in: USD)
      buyCurrency{symbol}

      sellAmount
      sellCurrency{symbol}
      sellAmountInUsd:sellAmount(in: USD)

      tradeAmount(in: USD)
    }
  }
}
"""
result = run_query(query)  # Execute the query
print ('Result - {}'.format(result))

Result - {'data': {'ethereum': {'dexTrades': None}}, 'errors': [{'message': 'ActiveRecord::ActiveRecordError: Response code: 500:\nCode: 241, e.displayText() = DB::Exception: Memory limit (for query) exceeded: would use 29.80 GiB (attempt to allocate chunk of 4718592 bytes), maximum: 29.80 GiB: While executing AggregatingTransform (version 20.8.11.17 (official build))\n', 'locations': [{'line': 4, 'column': 5}], 'path': ['ethereum', 'dexTrades'], 'error_type': 'server'}]}
